<p> IDE: Colab </p>

In [ ]:
import io,sys
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds
from keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [ ]:
def get_batch_data ():

  (train_data,test_data),info = tfds.load ('imdb_reviews/subwords8k',split=(tfds.Split.TRAIN,tfds.Split.TEST),with_info=True,as_supervised=True)
  encoder = info.features['text'].encoder
  padded_shapes = ([None],())
  train_batches = train_data.shuffle(1000).padded_batch(10, padded_shapes = padded_shapes)
  test_batches = test_data.shuffle(1000).padded_batch(10, padded_shapes = padded_shapes)

  return train_batches,test_batches,encoder


In [ ]:
def get_model (encoder, embedding_dim = 16):
  
  model = keras.Sequential ([layers.Embedding(encoder.vocab_size,embedding_dim),
                           layers.GlobalAveragePooling1D(),
                           layers.Dense(1,activation = 'sigmoid') ])
  model.compile(optimizer = 'adam',loss='binary_crossentropy',metrics=['accuracy'])
  history = model.fit(train_batches,epochs=10, validation_data=test_batches,validation_steps=20)  
  return model


In [ ]:
def plot_data (history) :
  
  history_dict = history.history
  acc = history_dict['accuracy']
  val_acc = history_dict ['accuracy']
  epochs = range (1,len (acc) + 1)
  plt.figure (figsize=(12,9))
  plt.figure (figsize=(12,9))
  plt.plot (epochs,acc,'bo',label='train acc')
  plt.plot (epochs, val_acc,'b', label = 'Validation acc')
  plt.title ('Training and validation accuracy')
  plt.xlabel('Epochs')
  plt.ylabel ('Accuracy')
  plt.legend (loc='lower right')
  plt.ylim ((0.5,1))
  plt.show ()

  return 

In [ ]:
def retrieve_embeddings (model,encoder):
  out_vectors = io.open ('vecs.tsv','w',encoding = 'UTF-8')
  out_metadata =  io.open ('meta.tsv','w',encoding = 'UTF-8')
  weights = model.layers[0].get_weights()[0]

  for num,word in enumerate (encoder.subwords):
    vec = weights[num+1]
    out_metadata.write(word +'\n')
    out_vectors.write ('\t'.join([str(x) for x in vec ]) + '\n')
  out_metadata.close()
  out_vectors.close()
  
  return
  

In [ ]:
train_batches, test_batches,encoder = get_batch_data()
model = get_model (encoder)
history = model.fit(train_batches,epochs=10, validation_data=test_batches,validation_steps=20)
#plot_history (history)
retrieve_embeddings (model,encoder)

Epoch 1/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.6334 - accuracy: 0.6958 - val_loss: 0.5248 - val_accuracy: 0.8050
Epoch 2/10
2500/2500 [==============================] - 15s 6ms/step - loss: 0.4617 - accuracy: 0.8414 - val_loss: 0.3819 - val_accuracy: 0.8950
Epoch 3/10
2500/2500 [==============================] - 15s 6ms/step - loss: 0.3623 - accuracy: 0.8802 - val_loss: 0.3701 - val_accuracy: 0.8800
Epoch 4/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.3076 - accuracy: 0.8970 - val_loss: 0.3731 - val_accuracy: 0.8600
Epoch 5/10
2500/2500 [==============================] - 16s 7ms/step - loss: 0.2726 - accuracy: 0.9077 - val_loss: 0.4000 - val_accuracy: 0.8650
Epoch 6/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.2489 - accuracy: 0.9162 - val_loss: 0.3410 - val_accuracy: 0.8700
Epoch 7/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.2273 - accuracy: 0.9234 - val_loss: 0.2585 - val_ac

NameError: ignored

In [ ]:
retrieve_embeddings (model,encoder)